## Making the EBcache
+ should work similar to EBscore with these differences:
    * no tumor bam is needed

In [ ]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
#  home = '/Users/mahtin'


somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "PON")

static = os.path.join(home, "Dropbox/Icke/Work/static")

## make EBcache matrix
+ to be used by bam2matrix in cache-mode

from ebcache import PON2matrix

EBconfig = {
    "temp_dir": os.path.join(ebdata, "temp"),
    "pon_path": pon_path,
    "zero_path": os.path.join(pon_path, "zero"),
    "bed_file": os.path.join(static, "bed_files/SureSelect/hg38/SS_HAEv7_hg38_Padded.bed"),
    "mawk_path": "../shell",
    "genome_split": f"{static}/genome/gatk/hg38/split",
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "use_cache": True
}
pon_list = "PON_chr19.txt"
chrom = "chr19"

PON2matrix(pon_list, chrom, EBconfig)

In [ ]:
chr19_df = pd.read_csv(os.path.join(pon_path, "matrix/chr19.pon.gz"), compression="gzip", sep="\t")
chr19_df

## computation has  to be done per PON string
+ allows use of same function for both PONcache and matrix2AB
+ allows optimal caching of the results

In [ ]:
from ebcache import PONmatrix2AB_multi

PONAB_config = {
    "fit_pen": 0.5,
    "threads": 8,
    "chunk_size":20000,
    "zero_file": os.path.join(pon_path, "zero.csv")
}
test_df =  chr19_df[:200000].copy()

chr19_AB_df = PONmatrix2AB_multi(chr19_df, config=PONAB_config)

In [ ]:
chr19_AB_df

### matrix to AB
+ look at the first 500k lines of AB-converted pon and try to see what comes most and what could be cached for faster computation
### caching the caching
+ trying to find common cachable AB parameters

In [ ]:
from ebcache import PONmatrix2AB_multi

lines = 100000

# use this but going through all letters and 
PONEB_config = {
    "fit_pen": 0.5,
    "threads": 8,
    "chunk_size": 1900
}

%time df = PONmatrix2AB_multi(chr19_df[:lines], config=PONEB_config)
# df.to_csv(os.path.join(pon_path, "ABcache/chr19.AB.gz"), sep="\t", compression="gzip", index=False)
df

### flatten the df by the letters

In [ ]:
# merge AB with matrix
merge = chr19_df[:lines].merge(df, left_index=True, right_index=True)

dfs = []
for letter in "AGCTID":
    print(letter)
    letter_df = merge.loc[:, [f"{letter}_x", "Depth", f"{letter}_y"]].rename({f"{letter}_x":"tumor", f"{letter}_y":"AB"}, axis=1)
    dfs.append(letter_df)
merge2 = pd.concat(dfs)
merge2

## split left and right

In [ ]:
ldfs = []
for l in [0,1]:
    ldf = pd.DataFrame()
    for col in ['tumor', 'Depth', 'AB']:
        ldf[col] = merge2[col].str.split("=").str[0]
    ldfs.append(ldf)
merge3 = pd.concat(ldfs)
merge3

### sort the splits by Depth

In [ ]:
def order(row):
    dl = [int(d) for d in row['Depth'].split("|")]
    row['Depth'] = "|".join([str(d) for d in sorted(dl)])
    row['tumor'] = "|".join([str(row['tumor'].split("|")[i]) for i in np.argsort(dl)])
    return row

merge4 = merge3.apply(order, axis=1).sort_values(['AB', 'Depth', 'tumor']).groupby(['Depth', 'tumor']).agg(['first', 'last', "count"])

### make stats

In [ ]:
merge5 = merge4['AB'].reset_index().sort_values(['count'], ascending=False).query('tumor == "0|0|0|0|0"')
merge5['freq'] = merge5['count'] / (lines * 12)
merge5['sumfreq'] = merge5['freq'].cumsum()
merge5

In [ ]:
merge5.query('tumor == "0|0|0|0|0"')

In [ ]:
merge5.to_csv(os.path.join(pon_path, "chr19.cache.cache.gz"), sep="\t", index=False, compression="gzip")
# merge7 = pd.read_csv(os.path.join(pon_path, "chr7.cache.cache.gz"), sep="\t", compression="gzip")